## Преобразование урлов

In [1]:
import pandas as pd
from gensim.models import Word2Vec
from tqdm.notebook import tqdm
import gc
import numpy as np

In [2]:
train = pd.read_csv('train', sep = '\t')
test = pd.read_csv('test', sep = '\t')

In [3]:
train.fillna(' ', inplace = True) 
test.fillna(' ', inplace = True) 

### Обрабатываем токены и урлы 

In [4]:
train['tokens_proc'] = train.tokens.apply(lambda t: ''.join([((x + ' ') * int(y)) for x,y in zip(t.split(' ')[::2], t.split(' ')[1::2])]) if t != ' ' else ' ')
train['urls_proc'] = train.urls_hashed.apply(lambda t: ''.join([((x + ' ') * int(y)) for x,y in zip(t.split(' ')[::2], t.split(' ')[1::2])]) if t != ' ' else ' ')

train['tokens_proc_no_count'] = train.tokens.apply(lambda t: ' '.join(t.split(' ')[::2]) if t != ' ' else ' ')
train['urls_proc_no_count'] = train.urls_hashed.apply(lambda t: ' '.join(t.split(' ')[::2]) if t != ' ' else ' ')

In [5]:
test['tokens_proc'] = test.tokens.apply(lambda t: ''.join([((x + ' ') * int(y)) for x,y in zip(t.split(' ')[::2], t.split(' ')[1::2])]) if t != ' ' else ' ')
test['urls_proc'] = test.urls_hashed.apply(lambda t: ''.join([((x + ' ') * int(y)) for x,y in zip(t.split(' ')[::2], t.split(' ')[1::2])]) if t != ' ' else ' ')

test['tokens_proc_no_count'] = test.tokens.apply(lambda t: ' '.join(t.split(' ')[::2]) if t != ' ' else ' ')
test['urls_proc_no_count'] = test.urls_hashed.apply(lambda t: ' '.join(t.split(' ')[::2]) if t != ' ' else ' ')

In [9]:
train.head()

,CLIENT_ID,RETRO_DT,tokens,DEF,urls_hashed,tokens_proc,urls_proc,tokens_proc_no_count,urls_proc_no_count
0,5909886,20200911,форм 1 мачеха 1 работы 1 пасынка 1 горячую 1 м...,0,7bbcc146c0ba4df814701ff7297b2205 1 ee0d49cbc99...,форм мачеха работы пасынка горячую массаж маче...,7bbcc146c0ba4df814701ff7297b2205 ee0d49cbc99c7...,форм мачеха работы пасынка горячую массаж маче...,7bbcc146c0ba4df814701ff7297b2205 ee0d49cbc99c7...
1,1385448,20210519,platiza 1 мобильные 2 sergej 1 code 3 отказов ...,1,1889e0cbd3e343e9521a07aa312257ac 2 8253d8d3a67...,platiza мобильные мобильные sergej code code c...,1889e0cbd3e343e9521a07aa312257ac 1889e0cbd3e34...,platiza мобильные sergej code отказов самовыво...,1889e0cbd3e343e9521a07aa312257ac 8253d8d3a6780...
2,1866195,20210913,sims 3 кредит 1 языке 1 разлива 1 огромное 1 о...,0,6e617b4bfdbef85d8b09ec08ac72f0f2 1 94309f5f116...,sims sims sims кредит языке разлива огромное о...,6e617b4bfdbef85d8b09ec08ac72f0f2 94309f5f11614...,sims кредит языке разлива огромное окна prosta...,6e617b4bfdbef85d8b09ec08ac72f0f2 94309f5f11614...
3,1587081,20210702,code 1 историй 1 scf 1 шаг 1 деньги 3 серый 1 ...,0,1e833434273e04ba76cfcfb4b48ad21b 3 aee71c8d18d...,code историй scf шаг деньги деньги деньги серы...,1e833434273e04ba76cfcfb4b48ad21b 1e833434273e0...,code историй scf шаг деньги серый авиабилеты t...,1e833434273e04ba76cfcfb4b48ad21b aee71c8d18d6b...
4,1400908,20210522,отказов 2 tovara 1 сергея 1 купальный 1 чёрный...,0,83e85bee0d81c0e9e5047e202cddf534 3 6e617b4bfdb...,отказов отказов tovara сергея купальный чёрный...,83e85bee0d81c0e9e5047e202cddf534 83e85bee0d81c...,отказов tovara сергея купальный чёрный деньги ...,83e85bee0d81c0e9e5047e202cddf534 6e617b4bfdbef...


### Добавляем веса для урлов в зависимости от их встречамости в таргете

In [ ]:
urls = [x.split(' ') for x in list(train['urls_proc'].values)]
urls_weights = pd.DataFrame(pd.Series(list(itertools.chain(*urls))).value_counts())

In [11]:
urls_weights = urls_weights.reset_index()
urls_weights.columns = ['url', 'cnt']
urls_weights

,url,cnt
0,0028c3b68113a97ccf75351d44d647cd,10865222
1,c9f124b819b7a2dbef3f0bcbf0330b9b,7877987
2,aee71c8d18d6b91b36bf599208a91777,5073539
3,c559cff1e7d1df756cb212a22311ea3b,3358783
4,7405aaece8dbd9b744b0752a7ecae526,2148156
...,...,...
622803,efd36340df6a49bb07f9b309df0357eb,1
622804,6919c84513c55739e641ce52a62d1996,1
622805,de3e78ef9bf3b57e5910a7fa81004cdb,1
622806,37785f18b3a1733996c99678068ce094,1


In [9]:
urls_pos = [x.split(' ') for x in list(train[train.DEF == 1]['urls_proc'].values)]
urls_weights_pos = pd.DataFrame(pd.Series(list(itertools.chain(*urls_pos))).value_counts())

In [12]:
urls_weights_pos = urls_weights_pos.reset_index()
urls_weights_pos.columns = ['url', 'cnt']

In [13]:
urls_weights_pos

,url,cnt
0,0028c3b68113a97ccf75351d44d647cd,2313485
1,c9f124b819b7a2dbef3f0bcbf0330b9b,1679947
2,aee71c8d18d6b91b36bf599208a91777,1168520
3,c559cff1e7d1df756cb212a22311ea3b,737409
4,7405aaece8dbd9b744b0752a7ecae526,457986
...,...,...
295476,d4ab274b0bd9dc840f1ab4044fca1279,1
295477,7bffc5fce3b5688f699df1e0e0c41910,1
295478,dab3677d220e0b2735836b3fb7cc8790,1
295479,c2d1014f8e731557b89c08a2677e1c1d,1


In [15]:
urls_weights_table = urls_weights.merge(urls_weights_pos, how = 'left', on = 'url')

In [17]:
urls_weights_table.fillna(0, inplace = True)
urls_weights_table['weight'] = urls_weights_table['cnt_y']/urls_weights_table['cnt_x']

In [18]:
urls_weights_table

,url,cnt_x,cnt_y,weight
0,0028c3b68113a97ccf75351d44d647cd,10865222,2313485.0,0.212926
1,c9f124b819b7a2dbef3f0bcbf0330b9b,7877987,1679947.0,0.213246
2,aee71c8d18d6b91b36bf599208a91777,5073539,1168520.0,0.230317
3,c559cff1e7d1df756cb212a22311ea3b,3358783,737409.0,0.219546
4,7405aaece8dbd9b744b0752a7ecae526,2148156,457986.0,0.213200
...,...,...,...,...
622803,efd36340df6a49bb07f9b309df0357eb,1,0.0,0.000000
622804,6919c84513c55739e641ce52a62d1996,1,0.0,0.000000
622805,de3e78ef9bf3b57e5910a7fa81004cdb,1,0.0,0.000000
622806,37785f18b3a1733996c99678068ce094,1,1.0,1.000000


Отбрасываем урлы, которых либо слишком мало, либо не слишком репрезентативны

In [29]:
urls_weights_table_final = urls_weights_table[(urls_weights_table.cnt_x > 100) & 
                                              ((urls_weights_table.weight < train.DEF.mean() - 0.05) | 
                                               (urls_weights_table.weight > train.DEF.mean() + 0.05))]

In [32]:
urls_weights_table_final = urls_weights_table_final.sort_values('weight').reset_index(drop=True)
urls_weights_table_final.to_csv('urls_weights.csv', index = False)
urls_weights_table_final

,url,cnt_x,cnt_y,weight
0,b9827649dbe19af5850b3693e60d6e77,162,0.0,0.0
1,b044a598ea01b4ff76e8950b3f216750,191,0.0,0.0
2,fee5b414938244fca1d20867b75a8d53,191,0.0,0.0
3,bc38e13599f3bc595e8dcda95aec468f,111,0.0,0.0
4,f4fe6cc37e82ee48ad7b1633ccdbfa10,111,0.0,0.0
...,...,...,...,...
20544,2d7662d719231a31af634bbc4f6ba60e,201,201.0,1.0
20545,7e3217b15879152eb655f58254a888b3,483,483.0,1.0
20546,3468c27a2e8bb801575d06a46eb3aa5f,159,159.0,1.0
20547,7496892a3e4f3cee7a445f2d9a449eac,146,146.0,1.0


### Преобразование urls в word2vec используя очищенные последовательности

In [ ]:
sec = np.load('sequences.npy', allow_pickle=True)

In [ ]:
urls_weights = pd.read_csv('urls_weights.csv')
urls_to_save = list(urls_weights.url.values)

In [ ]:
# Избавиться от урлов, которые исключили
sec_proc = []
for i in tqdm(sec):
    sec_proc.append(list(set(i) & set(urls_to_save)))

In [ ]:
with open('sec_proc.npy', 'wb') as f:
    np.save(f, sec_proc)

Word2vec

In [14]:
model = Word2Vec(sentences=sec_proc, 
                 vector_size=256, 
                 window=3, 
                 min_count=10, 
                 workers=15)

model.build_vocab(corpus_iterable=sec_proc, progress_per=10000)

In [15]:
model.train(corpus_iterable=sec_proc, 
            total_examples = model.corpus_count, 
            epochs=30, 
            report_delay=1)

(1574707030, 1921629930)

In [18]:
model.save("word2vec_256.model")

### Преобразование train и test

In [8]:
urls_weights = pd.read_csv('urls_weights.csv')
w2v_256 = Word2Vec.load("word2vec_256.model")

In [9]:
urls_weights_dict = {i: j for (i, j) in zip (urls_weights.url.values, urls_weights.weight.values)}
keys = urls_weights_dict.keys()
urls_to_use = set(urls_weights_dict.keys())

In [13]:
train['urls_emb_vw'] = train.urls_proc_no_count.apply(lambda t: np.sum([w2v_256.wv[i] * urls_weights_dict[i]
                                                                if (i in w2v_256.wv and i in urls_to_use and i != '' and i != [] )
                                                                else np.zeros(256) 
                                                                for i in list(set(t.split(' ')[:-1]) & urls_to_use)], axis = 0)  )

In [11]:
test['urls_emb_vw'] = test.urls_proc_no_count.apply(lambda t: np.sum([w2v_256.wv[i] * urls_weights_dict[i]
                                                                if (i in w2v_256.wv and i in urls_to_use and i != '' and i != [] )
                                                                else np.zeros(256) 
                                                                for i in list(set(t.split(' ')[:-1]) & urls_to_use)], axis = 0)  )

In [14]:
t = type(train['urls_emb_vw'][0])

In [15]:
emb_urls_train = [list(x) if type(x) == t else list(np.zeros(256)) for x in train['urls_emb_vw'].values]
emb_urls_train = pd.DataFrame(emb_urls_train)

In [16]:
emb_urls_test = [list(x) if type(x) == t else list(np.zeros(256)) for x in test['urls_emb_vw'].values]
emb_urls_test = pd.DataFrame(np.array(emb_urls_test))

In [17]:
emb_urls_test.columns = [str(i) for i in range(emb_urls_test.shape[1])]
emb_urls_train.columns = [str(i) for i in range(emb_urls_train.shape[1])]

In [21]:
emb_urls_test.to_feather('w2v_256_test')
emb_urls_train.to_feather('w2v_256_train')

feature generation 

In [12]:
train['urls_clean'] = train.urls_proc_no_count.apply(lambda t: ' '.join(list(set(t.split(' ')[:-1]) & urls_to_use)))
test['urls_clean'] = test.urls_proc_no_count.apply(lambda t: ' '.join(list(set(t.split(' ')[:-1]) & urls_to_use))) 

In [15]:
train.urls_clean[0]

'f49b1429e09e7185dad68e7e6f04e7ed 088d9cd775cbb207873d81c48f5c9218 e2afdb2c9457352689e6fa39b8bb6e3b 09344189eccdea6050b00d4b1878d101 9e8266a43f3c2bbf0fc60969ab7c31fa b1523d9d11a5e0052e4f80e0e8850587 ed5765b171ca635f97ffc0843f72155c'

In [17]:
train

,CLIENT_ID,RETRO_DT,tokens,DEF,urls_hashed,tokens_proc,urls_proc,tokens_proc_no_count,urls_proc_no_count,urls_clean
0,5909886,20200911,форм 1 мачеха 1 работы 1 пасынка 1 горячую 1 м...,0,7bbcc146c0ba4df814701ff7297b2205 1 ee0d49cbc99...,форм мачеха работы пасынка горячую массаж маче...,7bbcc146c0ba4df814701ff7297b2205 ee0d49cbc99c7...,форм мачеха работы пасынка горячую массаж маче...,7bbcc146c0ba4df814701ff7297b2205 ee0d49cbc99c7...,f49b1429e09e7185dad68e7e6f04e7ed 088d9cd775cbb...
1,1385448,20210519,platiza 1 мобильные 2 sergej 1 code 3 отказов ...,1,1889e0cbd3e343e9521a07aa312257ac 2 8253d8d3a67...,platiza мобильные мобильные sergej code code c...,1889e0cbd3e343e9521a07aa312257ac 1889e0cbd3e34...,platiza мобильные sergej code отказов самовыво...,1889e0cbd3e343e9521a07aa312257ac 8253d8d3a6780...,dc391a137297e7f1b8aa50a85b3b0f83 33b8dd65e21bb...
2,1866195,20210913,sims 3 кредит 1 языке 1 разлива 1 огромное 1 о...,0,6e617b4bfdbef85d8b09ec08ac72f0f2 1 94309f5f116...,sims sims sims кредит языке разлива огромное о...,6e617b4bfdbef85d8b09ec08ac72f0f2 94309f5f11614...,sims кредит языке разлива огромное окна prosta...,6e617b4bfdbef85d8b09ec08ac72f0f2 94309f5f11614...,0918801f328853571731d60745f25fde 69edacc7f0fbd...
3,1587081,20210702,code 1 историй 1 scf 1 шаг 1 деньги 3 серый 1 ...,0,1e833434273e04ba76cfcfb4b48ad21b 3 aee71c8d18d...,code историй scf шаг деньги деньги деньги серы...,1e833434273e04ba76cfcfb4b48ad21b 1e833434273e0...,code историй scf шаг деньги серый авиабилеты t...,1e833434273e04ba76cfcfb4b48ad21b aee71c8d18d6b...,64a7275a3ae205eb5220dbc838a74433 004ad420dae58...
4,1400908,20210522,отказов 2 tovara 1 сергея 1 купальный 1 чёрный...,0,83e85bee0d81c0e9e5047e202cddf534 3 6e617b4bfdb...,отказов отказов tovara сергея купальный чёрный...,83e85bee0d81c0e9e5047e202cddf534 83e85bee0d81c...,отказов tovara сергея купальный чёрный деньги ...,83e85bee0d81c0e9e5047e202cddf534 6e617b4bfdbef...,89cf7c4ae9175d3755a4a50e9c8d96b6 7a592fa533c42...
...,...,...,...,...,...,...,...,...,...,...
303772,6220063,20201211,морковь 1 вред 1 описание 1 калорийность 1 пол...,0,2c45709d8e05171bfb39b7528e97ac5f 1 0028c3b6811...,морковь вред описание калорийность польза свой...,2c45709d8e05171bfb39b7528e97ac5f 0028c3b68113a...,морковь вред описание калорийность польза свой...,2c45709d8e05171bfb39b7528e97ac5f 0028c3b68113a...,2c45709d8e05171bfb39b7528e97ac5f
303773,5543361,20200507,мотылёк 1 uploader 1 photo 1 провокатор 2 смс ...,0,30d2c325f9f511359a476c67e1130ef5 2 18394875e74...,мотылёк uploader photo провокатор провокатор с...,30d2c325f9f511359a476c67e1130ef5 30d2c325f9f51...,мотылёк uploader photo провокатор смс скачай m...,30d2c325f9f511359a476c67e1130ef5 18394875e7480...,18394875e7480cb47dc85ced87f0257e 492d66f190a09...
303774,224227891,20211218,чёрный 1 деньги 12 бытовой 1 серый 3 маршрутов...,0,e7f1dd5dbad8f9535e775b21bb0b9173 1 a4dba6829fa...,чёрный деньги деньги деньги деньги деньги день...,e7f1dd5dbad8f9535e775b21bb0b9173 a4dba6829fa5d...,чёрный деньги бытовой серый маршрутов турнир а...,e7f1dd5dbad8f9535e775b21bb0b9173 a4dba6829fa5d...,ecf7e77aaede267eaa6932a29b88ee6e 03ec8b6436b6f...
303775,5909478,20200911,kabelya 1 pesen 2 статья 2 опубликовала 1 купи...,0,ccbbab80c3bf2d7533e2ca1baed78073 1 503c50cb580...,kabelya pesen pesen статья статья опубликовала...,ccbbab80c3bf2d7533e2ca1baed78073 503c50cb5804c...,kabelya pesen статья опубликовала купить golf ...,ccbbab80c3bf2d7533e2ca1baed78073 503c50cb5804c...,5281f105e33f53f86edc703c53f4f06d efd163e6baf76...


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
def generate_features(df, column_name):
    """
    df - train или тест 
    column_name - колонка с строкой, в которой через пробел находятся урлы или токены очищенные от по весам
    """
    df['char_count'] = df[column_name].apply(len)
    df['word_count'] = df[column_name].apply(lambda x: len(x.split()))
    df['word_density'] = df['char_count'] / (df['word_count']+1)

    
    # BoW
    print('start CountVectorizer...')
    cvectorizer = CountVectorizer(max_features=1000)
    cvz = cvectorizer.fit_transform(df[column_name])

    # Latent Dirichlet Allocation
    print('start Latent Dirichlet Allocation...')
    lda_model = LatentDirichletAllocation(n_components=20, learning_method='online', random_state=42)
    X_topics = lda_model.fit_transform(cvz)
    
    X_topics = pd.DataFrame(X_topics)
    X_topics.columns = [f'lde_feature_{i}' for i in range(X_topics.shape[1])]

    df = pd.concat([df, X_topics], axis = 1)
    
    return df

In [27]:
train = generate_features(train, 'urls_clean')

start CountVectorizer...
start Latent Dirichlet Allocation...
